<a href="https://colab.research.google.com/github/h4ck4l1/datasets/blob/main/NLP_with_RNN_and_Attention/Bidirectional_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
import os,warnings
os.environ["TF_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")
from zipfile import ZipFile
import numpy as np
import tensorflow as tf
from tensorflow import keras
tf.get_logger().setLevel("ERROR")
import plotly.graph_objects as go

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
with tf.device("/job:localhost"):
    file_path = keras.utils.get_file(fname="/content/spa_en.zip",origin="https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",extract=True)
    with ZipFile(file_path,"r") as f:
        f.extractall("/content/spa_en")
    with open("/content/spa_en/spa-eng/spa.txt","r") as f:
        total_text = f.read()
    total_text = total_text.replace("¿","").replace("¡","")
    new_text = [line.split("\t") for line in total_text.splitlines()]
    en_text,es_text = zip(*new_text)

2638744/2638744 [==============================] - 0s 0us/step


In [ ]:
def get_layers(en_text,es_text,vocab_size=1000,seq_length=50):
    en_vec_layer = keras.layers.TextVectorization(vocab_size,output_sequence_length=seq_length)
    es_vec_layer = keras.layers.TextVectorization(vocab_size,output_sequence_length=seq_length)
    en_vec_layer.adapt(en_text)
    es_vec_layer.adapt([f"sos {s} eos" for s in es_text])
    return en_vec_layer,es_vec_layer

In [ ]:
def get_datasets(en_vec_layer,es_vec_layer,en_text,es_text,train_size):
    X_train = en_vec_layer(tf.constant(en_text[:train_size]))
    X_valid = en_vec_layer(tf.constant(en_text[train_size:]))
    X_dec_train = es_vec_layer(tf.constant([f"sos {s}" for s in es_text[:train_size]]))
    X_dec_valid = es_vec_layer(tf.constant([f"sos {s}" for s in es_text[train_size:]]))
    y_train = es_vec_layer(tf.constant([f"{s} eos" for s in es_text[:train_size]]))
    y_valid = es_vec_layer(tf.constant([f"{s} eos" for s in es_text[train_size:]]))
    return (X_train,X_dec_train),y_train,(X_valid,X_dec_valid),y_valid

In [ ]:
class Bidirectional_RNN(keras.Model):

    def __init__(self,vocab_size=1000,embed_size=128,**kwargs):

        super(Bidirectional_RNN,self).__init__(**kwargs)
        self.en_embed = keras.layers.Embedding(vocab_size,embed_size,mask_zero=True)
        self.es_embed = keras.layers.Embedding(vocab_size,embed_size,mask_zero=True)
        self.bidirectional_encoder = keras.layers.Bidirectional(keras.layers.LSTM(256,return_state=True))
        self.decoder = keras.layers.LSTM(512,return_sequences=True)
        self.Dense_out = keras.layers.Dense(vocab_size,"softmax")

    def call(self,inputs):

        '''
        Here the states len is 4 that is one forward-cell shortterm and longterm states
        and backward-cell shortterm and longterm states. and also the shape will be
        [train_size,256] we halved it because we will concat the two short states and two long states
        of the two rnn cells and the we get the normal concated short states and long states
        '''
        enc_inputs = inputs[0]
        dec_inputs = inputs[1]
        en_embed_out = self.en_embed(enc_inputs)
        es_embed_out = self.es_embed(dec_inputs)
        en_bidirec_output,*en_bidirec_states =  self.bidirectional_encoder(en_embed_out)
        all_states = [tf.concat(en_bidirec_states[::2],axis=-1),tf.concat(en_bidirec_states[1::2],axis=-1)]
        es_decoder_out = self.decoder(es_embed_out,initial_state=all_states)
        total_out = self.Dense_out(es_decoder_out)
        return total_out

In [ ]:
def piecewise(epoch,lr):
    if epoch < 6:
        return lr
    elif epoch < 10:
        return 5e-4
    else:
        return 5e-4 * tf.math.exp(-0.1695*(epoch-10))


with strategy.scope():
    train_size = 100_000
    valid_size = len(en_text) - train_size
    BATCH_SIZE = 50*8
    train_steps = train_size//BATCH_SIZE
    valid_steps = valid_size//BATCH_SIZE
    en_vec_layer,es_vec_layer = get_layers(en_text,es_text)
    X_train,y_train,X_valid,y_valid = get_datasets(en_vec_layer,es_vec_layer,en_text,es_text,train_size)
    bidirec_model = Bidirectional_RNN()
    exp_decay = keras.optimizers.schedules.ExponentialDecay(1e-2,decay_steps=train_steps,decay_rate=0.9)
    bidirec_model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=keras.optimizers.AdamW(learning_rate=5e-3),
        metrics=["accuracy"],
        steps_per_execution=25
    )
    lr_call = keras.callbacks.LearningRateScheduler(piecewise)

history = bidirec_model.fit(
    X_train,
    y_train,
    validation_data=(X_valid,y_valid),
    epochs=20,
    batch_size=BATCH_SIZE,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=[lr_call]
    )

print("\n\n")
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'],name="Train Loss",mode="lines"))
fig.add_trace(go.Scatter(y=history.history['val_loss'],name="Validation Loss",mode="lines"))
fig.show()

Epoch 1/20
250/250 [==============================] - 40s 160ms/step - loss: 3.1379 - accuracy: 0.4107 - val_loss: 2.8399 - val_accuracy: 0.4031 - lr: 0.0050
Epoch 2/20
250/250 [==============================] - 5s 19ms/step - loss: 1.6666 - accuracy: 0.6120 - val_loss: 2.2423 - val_accuracy: 0.4855 - lr: 0.0050
Epoch 3/20
250/250 [==============================] - 5s 19ms/step - loss: 1.2074 - accuracy: 0.6984 - val_loss: 2.0533 - val_accuracy: 0.5150 - lr: 0.0050
Epoch 4/20
250/250 [==============================] - 5s 19ms/step - loss: 0.9980 - accuracy: 0.7401 - val_loss: 2.0137 - val_accuracy: 0.5276 - lr: 0.0050
Epoch 5/20
250/250 [==============================] - 5s 18ms/step - loss: 0.8582 - accuracy: 0.7696 - val_loss: 2.0221 - val_accuracy: 0.5304 - lr: 0.0050
Epoch 6/20
250/250 [==============================] - 5s 19ms/step - loss: 0.7484 - accuracy: 0.7935 - val_loss: 2.0552 - val_accuracy: 0.5280 - lr: 0.0050
Epoch 7/20
250/250 [==============================] - 5s 19ms/

In [ ]:
def translate(sentence:str):
    translation = ""
    for word_id in range(50):
        X_inp = en_vec_layer(tf.constant([sentence]))
        X_dec_inp = es_vec_layer(tf.constant(["sos "+translation]))
        pred_proba = bidirec_model.predict((X_inp,X_dec_inp),verbose=0)[0,word_id]
        pred_ind = np.argmax(pred_proba)
        pred_word = es_vec_layer.get_vocabulary()[pred_ind]
        if pred_word == "eos":
            break
        translation += " " + pred_word
    return translation.strip()

In [ ]:
translate("I love Soccer")

'yo amo el fútbol'

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['accuracy'],name="Train Accuracy",mode="lines"))
fig.add_trace(go.Scatter(y=history.history['val_accuracy'],name="Validation Accuracy",mode="lines"))
fig.show()